In [1]:
import ee
import geemap
import geemap.colormaps as cm

In [3]:
cm.get_palette('terrain', 15)

['333399',
 '1b64ca',
 '0294fa',
 '00ba9d',
 '24d36d',
 '6de27c',
 'b6f08a',
 'ffff99',
 'dbd085',
 'b6a272',
 '92735e',
 '92736d',
 'b6a29d',
 'dbd0ce',
 'ffffff']

In [ ]:
Map = geemap.Map()

In [ ]:
Map

In [ ]:
# Map.add_legend(builtin_legend='ALOS_landforms')

In [ ]:
fc= ee.FeatureCollection('users/giswqs/public/countries')
# Map.addLayer(fc, {}, 'Countries')

In [ ]:
# rec = ee.Geometry.BBox(-179, -50, 179, 50)    
# Map.addLayer(rec, {}, 'Rec')    

In [ ]:
landforms = ee.Image('CSP/ERGo/1_0/Global/ALOS_landforms').select('constant')
landformsVis = {
  'min': 11.0,
  'max': 42.0,
  'palette': [
    '141414', '383838', '808080', 'EBEB8F', 'F7D311', 'AA0000', 'D89382',
    'DDC9C9', 'DCCDCE', '1C6330', '68AA63', 'B5C98E', 'E1F0E5', 'a975ba',
    '6f198c'
  ],
}

In [ ]:
Map.addLayer(landforms, landformsVis, 'Landforms')

In [ ]:
srtm = ee.Image("CGIAR/SRTM90_V4")
hillsahde = ee.Terrain.hillshade(srtm)

In [ ]:
# ned = ee.Image('USGS/3DEP/10m')
ned = ee.Image('USGS/SRTMGL1_003').select('elevation')

In [ ]:
vis = {'min': 0, 'max': 6000, 'palette': cm.palettes.dem}

In [ ]:
# Map.addLayer(hillsahde, {}, 'Hillshade')
Map.addLayer(srtm, vis, 'SRTM')

In [ ]:
Map.addLayer(ned, vis, "NED")

In [ ]:
srtm_blend = geemap.blend(top_layer=srtm, top_vis=vis)
Map.addLayer(srtm_blend, {}, 'SRTM Blend')

In [ ]:
ned_blend = geemap.blend(top_layer=ned, top_vis=vis)
Map.addLayer(ned_blend, {}, 'NED Blend')

In [ ]:
nlcd = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')
nlcd_vis = {'bands': ['landcover']}
Map.addLayer(nlcd, nlcd_vis, 'NLCD')

In [ ]:
diff = srtm.subtract(ned)

In [ ]:
Map.addLayer(diff, {'min':-20, 'max': 20, 'palette': cm.palettes.coolwarm}, 'Diff')

In [ ]:
# geemap.image_mean_value(diff, scale=500).getInfo()

In [ ]:
Map

In [ ]:
df = geemap.image_stats_by_zone(diff, landforms, reducer='mean')
df

In [ ]:
labels = [
        "Peak/ridge (warm)",
        "Peak/ridge",
        "Peak/ridge (cool)",
        "Mountain/divide",
        "Cliff",
        "Upper slope (warm)",
        "Upper slope",
        "Upper slope (cool)",
        "Upper slope (flat)",
        "Lower slope (warm)",
        "Lower slope",
        "Lower slope (cool)",
        "Lower slope (flat)",
        "Valley",
        "Valley (narrow)",]

In [ ]:
df['label'] = labels

In [ ]:
df

In [ ]:
landform_fc = landforms.reduceToVectors(**{'geometry': Map.user_roi, 'scale':1000, })

In [ ]:
# Map.addLayer(landform_fc, {}, 'Landform Vector')

In [ ]:
result = diff.reduceRegions(**{
    'collection': landform_fc,
    'reducer': ee.Reducer.mean(),
    'scale': 100

})

In [ ]:
c22 = diff.updateMask(landforms.eq(22)).selfMask()

In [ ]:
Map.addLayer(c22, {}, 'C22')

In [ ]:
from geemap.datasets import DATA

In [ ]:
DATA.users_giswqs_public_us_states

In [ ]:
fc= ee.FeatureCollection('users/giswqs/public/countries')

In [ ]:
# geemap.zonal_stats(c22, fc, 'stats_mean.csv', statistics_type='MEAN')

In [ ]:
geom = landforms.geometry().bounds()

In [ ]:
# values = landforms.reduceRegions(**{
#     'reducer': ee.Reducer.frequencyHistogram(),
#     'collection': ee.FeatureCollection([ee.Feature(geom)]),
#     'scale': 1000
# })

In [ ]:
# values.getInfo()

In [ ]:
values = geemap.image_value_list(landforms, region=geom, return_hist=False)

In [ ]:
values.getInfo()

In [ ]:
result = geemap.image_stats_by_zone(diff, landforms, reducer='mean')
result

In [ ]:
geemap.image_stats_by_zone(diff, nlcd, reducer='maximum')

In [ ]:
result.aggregate_array('zone').getInfo()

In [ ]:
result.aggregate_array('elevation').getInfo()

In [ ]:
result.toDictionary(['elevation']).getInfo()

In [ ]:
isinstance(ee.Reducer.histogram(), ee.Reducer)

In [ ]:
geom.getInfo()

In [ ]:
result = c22.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'scale': 1000,
    'geometry': Map.user_roi

})

In [ ]:
result.getInfo()

In [ ]:
ee.Image().set(result).set({'class': 22}).getInfo()